In [ ]:
from datetime import datetime
from transformers import AutoTokenizer
import transformers
import torch
from tqdm import tqdm
import pandas as pd


try:
    from sn_script.config import (
        Config,
        binary_category_name,
        category_name,
        subcategory_name,
        random_seed,
        half_number,
        model_type,
    )
except ModuleNotFoundError:
    import sys

    sys.path.append(".")
    from src.sn_script.config import (
        Config,
        binary_category_name,
        category_name,
        subcategory_name,
        random_seed,
        half_number,
        model_type,
    )

# pandasのprogress_applyを使うために必要
tqdm.pandas()


# プロンプト作成用の引数
LLM_ANOTATION_CSV_PATH = (
    Config.target_base_dir / f"{model_type}_500game_{half_number}_llm_annotation.csv"
)